# 📘 Notebook 01: Data Collection
# Emotional Geography of Books – Shraddha

This notebook collects and processes data related to books and authors.
It fetches author metadata, extracts country and gender information,
and enriches the author data with this additional information.

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
from time import sleep, time
from utils.data_preprocessing import load_all_books, clean_books
from utils.author_metadata import enrich_books_with_authors

In [2]:
# Load all books from the raw folder
df_all = load_all_books()
df_all.head()

📂 Found 5 Goodreads files.


📂 Loading Goodreads files: 100%|██████████| 5/5 [00:00<00:00, 105.30it/s]

📚 Total books loaded: 1015


,title,author,link,rating,ratings_count,description,published_year
0,The Midnight Library,Matt Haig,https://www.goodreads.com/book/show/52578297-t...,3.98,2m ratings,Between life and death there is a library.When...,2020
127,The Happiest Man on Earth,Eddie Jaku,https://www.goodreads.com/book/show/53239311-t...,4.62,111k ratings,Life can be beautiful if you make it beautiful...,2020
128,"The City We Became (Great Cities, #1)",N.K. Jemisin,https://www.goodreads.com/book/show/42074525-t...,3.85,75.5k ratings,Five New Yorkers must come together in order t...,2020
129,"Throttled (Dirty Air, #1)",Lauren Asher,https://www.goodreads.com/book/show/206023355-...,3.77,233k ratings,NoahMaya Alatorre is the sister of my teammate...,2020
130,If It Bleeds,Stephen King,https://www.goodreads.com/book/show/46015758-i...,3.98,114k ratings,If it Bleeds is a collection of four new novel...,2020


### Basic EDA

In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1015 entries, 0 to 1014
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1015 non-null   object 
 1   author          1015 non-null   object 
 2   link            1015 non-null   object 
 3   rating          1015 non-null   float64
 4   ratings_count   1015 non-null   object 
 5   description     1015 non-null   object 
 6   published_year  1015 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 63.4+ KB


In [5]:
df_all.describe(include="all")

,title,author,link,rating,ratings_count,description,published_year
count,1015,1015,1015,1015.000000,1015,1015,1015.000000
unique,998,600,1000,NaN,596,1000,NaN
top,"The Love Wager (Mr. Wrong Number, #2)",Freida McFadden,https://www.goodreads.com/book/show/60487511-t...,NaN,1m ratings,Hallie Piper is turning over a new leaf. After...,NaN
freq,2,18,2,NaN,16,2,NaN
mean,NaN,NaN,NaN,3.981429,NaN,NaN,2022.014778
std,NaN,NaN,NaN,0.275918,NaN,NaN,1.409596
min,NaN,NaN,NaN,2.790000,NaN,NaN,2020.000000
25%,NaN,NaN,NaN,3.790000,NaN,NaN,2021.000000
50%,NaN,NaN,NaN,4.000000,NaN,NaN,2022.000000
75%,NaN,NaN,NaN,4.170000,NaN,NaN,2023.000000


In [7]:
#Count number of rows per published_year
df_all["published_year"].value_counts()

published_year
2023    215
2020    200
2021    200
2022    200
2024    200
Name: count, dtype: int64

In [8]:
df_all.duplicated().sum()

15

In [3]:
df = df_all.copy()

In [4]:
df = clean_books(df)

In [8]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
title,1000,998,The Wife Upstairs,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
author,1000,600,Freida McFadden,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
link,1000,1000,https://www.goodreads.com/book/show/52578297-t...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rating,1000.0,NaN,NaN,NaN,3.98327,0.275731,2.79,3.79,4.0,4.17,4.76
ratings_count,1000.0,NaN,NaN,NaN,201622.796,262069.491392,11200.0,76600.0,116000.0,202250.0,3000000.0
description,1000,1000,Between life and death there is a library.When...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
published_year,1000.0,NaN,NaN,NaN,2022.0,1.414921,2020.0,2021.0,2022.0,2023.0,2024.0
author_first,1000,429,jennifer,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
source,1000,1,Goodreads,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# 🧠 Enrich with gender
df = enrich_books_with_authors(df)

# 🔍 Preview
df[["author", "author_gender"]].head()

⚠️ NamSor failed for 'Jessica  Joyce': 503 Server Error:  for url: https://v2.namsor.com/NamSorAPIv2/api2/json/genderFull/Jessica%20%20Joyce
⚠️ NamSor failed for 'Bethany Joy Lenz': ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
⚠️ NamSor failed for 'Callie Hart': HTTPSConnectionPool(host='v2.namsor.com', port=443): Max retries exceeded with url: /NamSorAPIv2/api2/json/genderFull/Callie%20Hart (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x13ee6cfd0>: Failed to establish a new connection: [Errno 61] Connection refused'))
⚠️ NamSor failed for 'Navessa Allen': HTTPSConnectionPool(host='v2.namsor.com', port=443): Max retries exceeded with url: /NamSorAPIv2/api2/json/genderFull/Navessa%20Allen (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x134ab9f90>: Failed to establish a new connection: [Errno 61] Connection refused'))
⚠️ NamSor failed for 'Lev Grossman': HTTPSConnectionPool(host='v2.namsor.com

NameError: name 'namsor_cache' is not defined

In [ ]:
df["author_gender"].value_counts()

author_gender
female     586
unknown    312
male       102
Name: count, dtype: int64

In [ ]:
df["gender_source"].value_counts()

#### Feature Engineering: Get Author's Country

In [117]:
#We have 301 authors without gender after scraping goodreads. So we will use an API to guess the author's gender, 
# failing which we will call it unknown/non-binary
# 1) Prepare gender_source col
df_enriched["gender_source"] = df_enriched["author_gender"].apply(
    lambda g: "goodreads" if g in ("male", "female") else "unknown"
)

# 2) Helper to call NamSor
def query_namsor(name: str = "") -> dict:
    """
    Query NamSor with a first & last name.
    Returns JSON with keys 'gender', 'probabilityCalibrated', etc.
    """
    headers = {
        "X-API-KEY": NAMSOR_API_KEY,
        "Accept": "application/json"    
    }
    url = NAMSOR_URL + name
    resp = requests.request("GET", url, headers=headers)
    resp.raise_for_status()
    return resp.json()

# 3) Identify authors needing a NamSor lookup
mask_unknown = df_enriched["author_gender"] == "unknown"
unknown_authors = (
    df_enriched[mask_unknown]
    .loc[:, ["author", "link"]]  
    .drop_duplicates("author")
    .reset_index(drop=True)
)
# 4) Call NamSor for each unique unknown author
namsor_cache = {}
for _, row in unknown_authors.iterrows():
    name = row["author"].strip()
    try:
        result = query_namsor(name=name)
        gender = result.get("likelyGender", "unknown")              # 'male'/'female'/'unknown'
        confidence = result.get("probabilityCalibrated", 0.0)       # between 0 and 1
    except Exception as e:
        print(f"⚠️ NamSor failed for {name!r}: {e}")
        gender, confidence = "unknown", 0.0

    # only trust if above threshold
    if confidence < 0.85:
        gender = "unknown/non-binary"

    namsor_cache[name] = {"gender": gender, "confidence": confidence}
    sleep(0.5)  # respect rate limits

# 5) Apply the NamSor results back to your main df
def fill_from_namsor(row):
    if row["author_gender"] == "unknown":
        meta = namsor_cache.get(row["author"], {"gender":"unknown/non-binary","confidence":0})
        gender = meta["gender"]
        if gender in ("male", "female"):
            row["author_gender"] = gender
            row["gender_source"] = "namsor"
        else:
            # still unknown or low-confidence
            row["author_gender"] = "unknown/non-binary"
            row["gender_source"] = "namsor"
    return row

df_final = df_enriched.apply(fill_from_namsor, axis=1)

# 6) Inspect how many remain unknown
print(df_final["author_gender"].value_counts())
print(df_final["gender_source"].value_counts())


author_gender
female                615
unknown/non-binary    230
male                  155
Name: count, dtype: int64
gender_source
goodreads    699
namsor       301
Name: count, dtype: int64


In [120]:
#Show stats of the unknown authors
df_final[df_final["author_gender"] == "unknown/non-binary"].describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
title,230,230,The Guest List,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
author,230,124,Ali Hazelwood,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
link,230,230,https://www.goodreads.com/book/show/52656911-t...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rating,230.0,NaN,NaN,NaN,3.957391,0.272232,3.15,3.75,3.97,4.1575,4.76
ratings_count,230.0,NaN,NaN,NaN,205145.217391,252450.306319,28900.0,76825.0,120000.0,215000.0,2000000.0
description,230,230,A wedding celebration turns dark and deadly in...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
published_year,230.0,NaN,NaN,NaN,2022.130435,1.373624,2020.0,2021.0,2022.0,2023.0,2024.0
author_first,230,115,ali,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
source,230,1,Goodreads,230,NaN,NaN,NaN,NaN,NaN,NaN,NaN
author_country,230,36,,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
df_final.to_csv("../data/processed/clean_books_2020_2024.csv", index=False)
print("✅ Saved cleaned data to data/processed/clean_books_2020_2024.csv")

✅ Saved cleaned data to data/processed/clean_books_2020_2024.csv
